In [1]:
import pandas as pd
import numpy as np
import os
import csv
import arcpy
from scipy.interpolate import griddata
from arcpy import env

In [2]:
# Define the base directory for the files
base_dir = "C:/Users/betebari/Documents/C2VSim_Texture/Aq-Params/Grouped_Results"

# Define the layer file paths relative to the base directory
layer_files = [
    f"{base_dir}/Grouped_Averages_L{i}.csv" for i in range(1, 5)
]

# Initialize an empty dictionary to store DataFrames
layer_dataframes = {}

# Loop through the files and load the data into DataFrames
for i, file in enumerate(layer_files, start=1):
    layer_dataframes[f"L{i}"] = pd.read_csv(file)

# Access the first DataFrame as an example
layer_dataframes["L1"].head()

,WellName,X,Y,Avg_Coarse,Avg_Kxy,Avg_Kv,Avg_Ss,Avg_Sy
0,1,578943.523506,4.395457e+06,49.345973,0.0001,0.230451,0.001553,10.847443
1,1,620122.947435,4.320000e+06,42.388978,0.0001,0.176190,0.001235,17.056492
2,1,713861.829177,4.107089e+06,35.072852,0.0001,0.182915,0.001195,13.817648
3,1,819457.149950,3.899480e+06,30.757834,0.0001,0.234428,0.001564,15.644755
4,10,587088.229224,4.280778e+06,37.165659,0.0001,0.203169,0.001576,11.186811


In [3]:
# Set the workspace
env.workspace = "C:/Users/betebari/Documents/C2VSim_Texture/Aq-Params/finalProduct"
env.overwriteOutput = True

In [4]:
layer_dataframes["L4"].head()

,WellName,X,Y,Avg_Coarse,Avg_Kxy,Avg_Kv,Avg_Ss,Avg_Sy
0,10004,723959.916934,4.177149e+06,10.231788,0.0001,6.821192e-08,6.821192e-07,2.046358
1,10071,800910.162596,4.082171e+06,36.415929,0.0001,2.501213e-01,8.607788e-04,22.765487
2,10072,800495.833418,4.081557e+06,5.000000,0.0001,5.000000e-04,2.500000e-03,3.000000
3,10083,740406.153000,4.003619e+06,53.268608,0.0001,3.722961e-01,9.017476e-04,20.409385
4,10098,781237.370694,4.090744e+06,39.208478,0.0001,3.111752e-01,1.223373e-03,20.403114


# IDW approach operation

In [14]:
from arcpy.sa import Idw  # Import the IDW function from Spatial Analyst

# Set the environment to allow overwriting existing files
arcpy.env.overwriteOutput = True

# Define the shapefile path
boundary_shapefile = "C:/Users/betebari/Documents/C2VSim_Texture/OSWCR/central_val_buf_5mil_utm10n.shp"

# Load the shapefile as a feature layer
arcpy.MakeFeatureLayer_management(boundary_shapefile, "boundary_layer")

# Get the extent of the boundary layer
extent = arcpy.Describe("boundary_layer").extent
arcpy.env.extent = extent  # Set the environment extent
arcpy.env.mask = "boundary_layer"  # Set the environment mask
arcpy.env.outputCoordinateSystem = arcpy.SpatialReference(26910)  # UTM Zone 10N

# Parameters to process
parameters = ['Avg_Coarse', 'Avg_Kxy', 'Avg_Kv', 'Avg_Ss', 'Avg_Sy']

# Ensure output folder exists
output_folder = "C:/Users/betebari/Documents/C2VSim_Texture/Aq-Params/finalProduct"
if not os.path.exists(output_folder):
    os.makedirs(output_folder)
    print(f"Created output folder: {output_folder}")

# Check if Spatial Analyst extension is available and check it out
if arcpy.CheckExtension("Spatial") == "Available":
    arcpy.CheckOutExtension("Spatial")
    print("Spatial Analyst extension checked out")
else:
    raise RuntimeError("Spatial Analyst extension not available")

# Loop through layers and parameters using the preloaded DataFrames
for layer, dataframe in layer_dataframes.items():
    for parameter in parameters:
        # Filter the DataFrame for relevant data (assume columns 'X', 'Y', and the parameter exist)
        filtered_df = dataframe[['X', 'Y', parameter]]

        # Save the filtered DataFrame to a temporary CSV file
        coords_csv = f"{output_folder}/Temp_{layer}_{parameter}.csv"
        filtered_df.to_csv(coords_csv, index=False)
        
        # Create a temporary XY event layer
        xy_event_layer = f"TempLayer_{layer}_{parameter}"
        arcpy.management.MakeXYEventLayer(coords_csv, 'X', 'Y', xy_event_layer, arcpy.env.outputCoordinateSystem)

        # Define output raster path
        raster_output = f"{output_folder}/{layer}_{parameter}_interp.tif"

        # Delete the existing raster if it exists
        if os.path.exists(raster_output):
            os.remove(raster_output)

        # Perform Inverse Distance Weighted (IDW) interpolation
        cell_size = 500  # Adjust based on your required grid resolution
        power = 2  # Default power value for IDW
        idw_result = Idw(xy_event_layer, parameter, cell_size, power)

        # Save the interpolated raster (overwrite if necessary)
        idw_result.save(raster_output)
        print(f"IDW interpolation complete for {parameter} in {layer}. Raster saved at {raster_output}")

# Check-in Spatial Analyst extension after completion
arcpy.CheckInExtension("Spatial")
print("All IDW interpolations completed.")

Spatial Analyst extension checked out
IDW interpolation complete for Avg_Coarse in L1. Raster saved at C:/Users/betebari/Documents/C2VSim_Texture/Aq-Params/finalProduct/L1_Avg_Coarse_interp.tif
IDW interpolation complete for Avg_Kxy in L1. Raster saved at C:/Users/betebari/Documents/C2VSim_Texture/Aq-Params/finalProduct/L1_Avg_Kxy_interp.tif
IDW interpolation complete for Avg_Kv in L1. Raster saved at C:/Users/betebari/Documents/C2VSim_Texture/Aq-Params/finalProduct/L1_Avg_Kv_interp.tif
IDW interpolation complete for Avg_Ss in L1. Raster saved at C:/Users/betebari/Documents/C2VSim_Texture/Aq-Params/finalProduct/L1_Avg_Ss_interp.tif
IDW interpolation complete for Avg_Sy in L1. Raster saved at C:/Users/betebari/Documents/C2VSim_Texture/Aq-Params/finalProduct/L1_Avg_Sy_interp.tif
IDW interpolation complete for Avg_Coarse in L2. Raster saved at C:/Users/betebari/Documents/C2VSim_Texture/Aq-Params/finalProduct/L2_Avg_Coarse_interp.tif
IDW interpolation complete for Avg_Kxy in L2. Raster sav

# Running Fill Function to smooth out -9999 and zeros (https://desktop.arcgis.com/en/arcmap/10.3/tools/spatial-analyst-toolbox/how-fill-works.htm

In [19]:
from arcpy.sa import Fill  # Import the Fill function from Spatial Analyst

# Ensure Spatial Analyst extension is available and check it out
if arcpy.CheckExtension("Spatial") == "Available":
    arcpy.CheckOutExtension("Spatial")
    print("Spatial Analyst extension checked out")
else:
    raise RuntimeError("Spatial Analyst extension is not available.")

# Boundary shapefile to set environment
boundary_shapefile = r"C:\Users\betebari\Documents\C2VSim_Texture\OSWCR\central_val_buf_5mil_utm10n.shp"

# Verify the shapefile exists
if not arcpy.Exists(boundary_shapefile):
    raise FileNotFoundError(f"Boundary shapefile not found: {boundary_shapefile}")

# Load the shapefile as a feature layer
arcpy.MakeFeatureLayer_management(boundary_shapefile, "boundary_layer")

# Set environment settings
arcpy.env.extent = arcpy.Describe("boundary_layer").extent  # Set the environment extent
arcpy.env.mask = "boundary_layer"  # Set the environment mask
arcpy.env.outputCoordinateSystem = arcpy.SpatialReference(26910)  # UTM Zone 10N

# Folder containing the IDW output rasters
output_folder = r"C:\Users\betebari\Documents\C2VSim_Texture\Aq-Params\finalProduct"

# Output folder for filled rasters
filled_folder = f"{output_folder}/FilledRasters"
if not os.path.exists(filled_folder):
    os.makedirs(filled_folder)
    print(f"Created filled rasters folder: {filled_folder}")

# List all rasters in the output folder
rasters = [os.path.join(output_folder, f) for f in os.listdir(output_folder) if f.endswith(".tif")]

# Apply Fill to each raster
for raster in rasters:
    raster_name = os.path.basename(raster)
    filled_raster = os.path.join(filled_folder, raster_name)

    # Perform Fill operation
    try:
        print(f"Processing {raster_name}...")
        filled_result = Fill(raster)
        
        # Save the filled raster
        filled_result.save(filled_raster)
        print(f"Filled raster saved to: {filled_raster}")
    except Exception as e:
        print(f"Error processing {raster_name}: {e}")

# Check-in Spatial Analyst extension
arcpy.CheckInExtension("Spatial")
print("All rasters processed and filled.")

Spatial Analyst extension checked out
Processing Idw_Temp_L1_Avg1.tif...
Filled raster saved to: C:\Users\betebari\Documents\C2VSim_Texture\Aq-Params\finalProduct/FilledRasters\Idw_Temp_L1_Avg1.tif
Processing L1_Avg_Coarse_interp.tif...
Filled raster saved to: C:\Users\betebari\Documents\C2VSim_Texture\Aq-Params\finalProduct/FilledRasters\L1_Avg_Coarse_interp.tif
Processing L1_Avg_Kv_interp.tif...
Filled raster saved to: C:\Users\betebari\Documents\C2VSim_Texture\Aq-Params\finalProduct/FilledRasters\L1_Avg_Kv_interp.tif
Processing L1_Avg_Kxy_interp.tif...
Filled raster saved to: C:\Users\betebari\Documents\C2VSim_Texture\Aq-Params\finalProduct/FilledRasters\L1_Avg_Kxy_interp.tif
Processing L1_Avg_Ss_interp.tif...
Filled raster saved to: C:\Users\betebari\Documents\C2VSim_Texture\Aq-Params\finalProduct/FilledRasters\L1_Avg_Ss_interp.tif
Processing L1_Avg_Sy_interp.tif...
Filled raster saved to: C:\Users\betebari\Documents\C2VSim_Texture\Aq-Params\finalProduct/FilledRasters\L1_Avg_Sy_int

# I create a shapfile for C2VSimFG nodes

In [22]:
# Define the path to the CSV file
csv_path = "C:/c2vsimfg1.5/C2VSimFG_node_layering_assigned.csv"

# Define the output shapefile path
output_shapefile = "C:/Users/betebari/Documents/C2VSim_Texture/Aq-Params/C2VSimFG_Nodes.shp"

# Define the spatial reference (e.g., UTM Zone 10N, EPSG: 26910)
spatial_ref = arcpy.SpatialReference(26910)  # Replace with your actual spatial reference

# Make an XY event layer from the CSV
temp_layer = "Temp_XY_Layer"  # Use a unique name for the temporary event layer
arcpy.management.MakeXYEventLayer(
    table=csv_path,
    in_x_field="X",  # Replace with the actual column name for X coordinates in your CSV
    in_y_field="Y",  # Replace with the actual column name for Y coordinates in your CSV
    out_layer=temp_layer,
    spatial_reference=spatial_ref
)

# Check if the output folder exists, and create it if it does not
output_folder = os.path.dirname(output_shapefile)
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Convert the event layer to a shapefile
arcpy.management.CopyFeatures(
    in_features=temp_layer,
    out_feature_class=output_shapefile
)

# Print success message
print(f"Shapefile created successfully at: {output_shapefile}")

Shapefile created successfully at: C:/Users/betebari/Documents/C2VSim_Texture/Aq-Params/C2VSimFG_Nodes.shp


# run this Function: EXTRACT VALUES TO POINTS with ArcPy

In [25]:
# Set environment to allow overwriting
arcpy.env.overwriteOutput = True

# Input point shapefile
output_shapefile = "C:/Users/betebari/Documents/C2VSim_Texture/Aq-Params/C2VSimFG_Nodes.shp"

# Folder containing raster files
output_folder = "C:/Users/betebari/Documents/C2VSim_Texture/Aq-Params/finalProduct"

# Ensure the final product folder exists
if not os.path.exists(output_folder):
    os.makedirs(output_folder)
    print(f"Created final product folder: {output_folder}")

# Parameters used in raster creation
parameters = ['Avg_Coarse', 'Avg_Kxy', 'Avg_Kv', 'Avg_Ss', 'Avg_Sy']
layers = ["L1", "L2", "L3", "L4"]

# Prepare the list of raster files for extraction
raster_files = []
field_mapping = []  # To store raster and field name mapping

for layer in layers:
    for parameter in parameters:
        raster_file = f"{output_folder}/{layer}_{parameter}_interp.tif"
        if arcpy.Exists(raster_file):
            raster_files.append(raster_file)
            field_name = f"{layer}_{parameter}"
            field_mapping.append((raster_file, field_name))
        else:
            print(f"Raster file not found: {raster_file}")

# Ensure Spatial Analyst extension is checked out
if arcpy.CheckExtension("Spatial") == "Available":
    arcpy.CheckOutExtension("Spatial")
    print("Spatial Analyst extension checked out.")
else:
    raise RuntimeError("Spatial Analyst extension is not available.")

# Perform Extract Multi Values to Points
try:
    print("Extracting raster values to points...")
    arcpy.sa.ExtractMultiValuesToPoints(output_shapefile, field_mapping, "NONE")
    print(f"Extraction complete. Values added to: {output_shapefile}")
finally:
    # Check in the Spatial Analyst extension
    arcpy.CheckInExtension("Spatial")
    print("Spatial Analyst extension checked in.")

Spatial Analyst extension checked out.
Extracting raster values to points...
Extraction complete. Values added to: C:/Users/betebari/Documents/C2VSim_Texture/Aq-Params/C2VSimFG_Nodes.shp
Spatial Analyst extension checked in.


In [27]:
# Input shapefile
shapefile = "C:/Users/betebari/Documents/C2VSim_Texture/Aq-Params/C2VSimFG_Nodes.shp"

# Output CSV file
output_csv = "C:/Users/betebari/Documents/C2VSim_Texture/Aq-Params/Aquifer_parameters_C2VSimFG2.0.csv"

# Read the attribute table
fields = [field.name for field in arcpy.ListFields(shapefile) if field.type not in ['Geometry']]  # Exclude geometry
with arcpy.da.SearchCursor(shapefile, fields) as cursor:
    # Write to CSV
    with open(output_csv, 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.writer(csvfile)
        # Write header
        writer.writerow(fields)
        # Write attribute rows
        for row in cursor:
            writer.writerow(row)

print(f"Attribute table of {shapefile} exported to {output_csv}")

Attribute table of C:/Users/betebari/Documents/C2VSim_Texture/Aq-Params/C2VSimFG_Nodes.shp exported to C:/Users/betebari/Documents/C2VSim_Texture/Aq-Params/Aquifer_parameters_C2VSimFG2.0.csv
